## **Web Scraping**

## Código utilizado para obtener los datos

In [ ]:
#imports
import requests
import bs4

Código de las funciones para la recogida de datos de los partidos

In [ ]:

#acceso y recogida de los datos de un partido
def matchdata(link):
  url = "https://fbref.com"+link
  html = requests.get(url, {}).text
  bs = bs4.BeautifulSoup(html, "lxml")

  #nombre equipos
  teamnames = []
  teamnames.append(bs.find('div', {'class':'scorebox'}).find_all('strong')[0].find('a').text)
  teamnames.append(bs.find('div', {'class':'scorebox'}).find_all('strong')[3].find('a').text)

  #minutos de los goles
  minhomegoals = []
  minawaygoals = []
  minhomeredcard = []
  minawayredcard = []
  home = bs.find('div', id="a")
  away = bs.find('div', id="b") 
  for x in home.find_all('div', {'class':'event_icon goal'}):
    event = []
    minute = "".join([x for x in x.parent.text if x.isdigit()])
    event.append(minute[0:2])
    event.append(x.parent.find_all('a')[0].text)
    minhomegoals.append(event)
  for x in away.find_all('div', {'class':'event_icon goal'}):
    event = []
    minute = "".join([x for x in x.parent.text if x.isdigit()])
    event.append(minute[0:2])
    event.append(x.parent.find_all('a')[0].text)
    minawaygoals.append(event)
  for x in home.find_all('div', {'class':'event_icon red_card'}):
    event = []
    minute = "".join([x for x in x.parent.text if x.isdigit()])
    event.append(minute[0:2])
    event.append(x.parent.find_all('a')[0].text)
    minhomeredcard.append(event)
  for x in away.find_all('div', {'class':'event_icon red_card'}):
    event = []
    minute = "".join([x for x in x.parent.text if x.isdigit()])
    event.append(minute[0:2])
    event.append(x.parent.find_all('a')[0].text)
    minawayredcard.append(event)

  #resultado
  resultado = []
  for x in bs.find_all('div', {'class':'score'}):
    resultado.append(x.text)

  #fecha del partido
  date = bs.find('div', {'class':'scorebox_meta'}).select("strong")[0].text

  #datos del partido
  homestats = []
  awaystats = []
  statsextra = bs.find('div', id="team_stats_extra").find_all('div')
  for x in statsextra:
    i = 0
    for y in x.find_all('div'):
      if i==3 or i==6 or i==9 or i==12 :
        homestats.append(y.text)
      elif i==5 or i==8 or i==11 or i==14 :
        awaystats.append(y.text)
      i = i+1

  #datos individuales de los jugadores locales
  homeplayers = bs.find_all('div', {'class':'table_outer_container'})[0].select("tbody")[0].select("tr")
  homeplayersdata = []
  for player in homeplayers:
    playerdata = []
    playerdata.append(player.find_all("th")[0].text.replace('\xa0\xa0\xa0', ''))
    if player.find_all("th")[0].text.find('\xa0\xa0\xa0') > -1:
      playerdata.append("no")
    else:
      playerdata.append("yes")
    for info in player.find_all("td"):
      playerdata.append(info.text)
    homeplayersdata.append(playerdata)

  #datos individuales de los porteros locales
  homegoalkeepers = bs.find_all('div', {'class':'table_outer_container'})[1].select("tbody")[0].select("tr")
  homegoalkeepersdata = []
  for player in homegoalkeepers:
    playerdata = []
    playerdata.append(player.find_all("th")[0].text)
    for info in player.find_all("td"):
      playerdata.append(info.text)
    homegoalkeepersdata.append(playerdata)

  #datos individuales de los jugadores visitantes
  awayplayers = bs.find_all('div', {'class':'table_outer_container'})[2].select("tbody")[0].select("tr")
  awayplayersdata = []
  for player in awayplayers:
    playerdata = []
    playerdata.append(player.find_all("th")[0].text.replace('\xa0\xa0\xa0', ''))
    if player.find_all("th")[0].text.find('\xa0\xa0\xa0') > -1:
      playerdata.append("no")
    else:
      playerdata.append("yes")
    for info in player.find_all("td"):
      playerdata.append(info.text)
    awayplayersdata.append(playerdata)

  #datos individuales de los porteros visitantes
  awaygoalkeepers = bs.find_all('div', {'class':'table_outer_container'})[3].select("tbody")[0].select("tr")
  awaygoalkeepersdata = []
  for player in awaygoalkeepers:
    playerdata = []
    playerdata.append(player.find_all("th")[0].text)
    for info in player.find_all("td"):
      playerdata.append(info.text)
    awaygoalkeepersdata.append(playerdata)

  data = [date, teamnames, resultado, homeplayersdata, awayplayersdata, homegoalkeepersdata, 
          awaygoalkeepersdata, minhomegoals, minawaygoals, minhomeredcard, minawayredcard]
  return data

#acceso a la web de cada uno de los partidos de una temporada
def seasondata(con, url, season): 
  html = requests.get(url, {}).text
  bs = bs4.BeautifulSoup(html, "lxml")
  enlaces = bs.find_all('a')
  i = 0
  for enlace in enlaces:   
    if "Match Report" in enlace:
      i = i+1
      link = enlace.get('href')
      data = matchdata(link)
      insertmatchdata(con, data, season, int((i+9)/10))



Código de las funciones para la obtención de todos los jugadores de todos los equipos

In [ ]:
def teamsseason(con, url, season): 
  html = requests.get(url, {}).text
  bs = bs4.BeautifulSoup(html, "lxml")
  
  i = 0
  for x in bs.find_all('td', {'data-stat':'squad_a'}):
    link = x.find('a').get('href')
    name = x.find('a').text
    data = teamsdata(link)
    insertplayers(con, data, name, season)
    i = i + 1 
    if i >= 10:
      break
  
  for x in bs.find_all('td', {'data-stat':'squad_b'}):
    link = x.find('a').get('href')
    name = x.find('a').text
    data = teamsdata(link)
    insertplayers(con, data, name, season)
    i = i + 1 
    if i >= 20:
      break



#obtiene los jugadores de todos los equipos
def teamsdata(link):
  url = "https://fbref.com"+link

  data = []

  html = requests.get(url, {}).text
  bs = bs4.BeautifulSoup(html, "lxml")
  
  table = bs.find_all('div', {'class':'table_outer_container'})[0].select("tbody")[0].select("tr")
  playersdata = []
  for player in table:
    playerinfo = []
    playerinfo.append(player.find_all("th")[0].text)   
    playerinfo.append(player.find_all("td")[1].text)
    playersdata.append(playerinfo)
    
  return playersdata


Funciones para sqlite

In [ ]:
import sqlite3
from google.colab import files

#obtener fichero db
def getfiledb():
  archive = files.upload()

#descargar fichero db
def downloadfiledb():
  files.download("dbfootballdata.db")

#abre el archivo db
def opendb():
  con = sqlite3.connect("dbfootballdata.db")
  return con

#cierra la db
def closedb(con):
  con.close()

#inserta todos los jugadores
def insertplayers(con, data, nameteam, season):
  cursor = con.cursor()
  abbrevteam = getabbrevteam(cursor, nameteam)
  for y in data:
    idplayer = getidplayer2(cursor, y[0])
    if len(idplayer) == 0:
      sql1 = 'INSERT INTO players (name, position) VALUES ("'+y[0]+'", "'+y[1]+'")'
      cursor.execute(sql1)
      idplayer = cursor.lastrowid
    else:
      idplayer = idplayer[0][0]
    sql2 = 'INSERT INTO playerteamseason (playerID, seasonID, abbrevteam) VALUES ("'+str(idplayer)+'","'+season+'","'+abbrevteam[0][0]+'")'
    cursor.execute(sql2)
    con.commit()

#inserta los datos de un partido
def insertmatchdata(con, data, season, week):
  cursor = con.cursor()
  local = getabbrevteam(cursor, data[1][0])
  visiting = getabbrevteam(cursor, data[1][1])
  idmatch = insertmatch(cursor, data[0], season, week, local[0][0], visiting[0][0], data[2][0], data[2][1])
  insertgoals(cursor, data[7], idmatch, data[1][0], season)
  insertgoals(cursor, data[8], idmatch, data[1][1], season)
  insertredcard(cursor, data[9], idmatch, data[1][0], season)
  insertredcard(cursor, data[10], idmatch, data[1][1], season)
  insertstats(cursor, idmatch, data[3], data[1][0], season)
  insertstats(cursor, idmatch, data[4], data[1][1], season)
  insertstatsgoalkeeper(cursor, idmatch, data[5], data[1][0], season)
  insertstatsgoalkeeper(cursor, idmatch, data[6], data[1][1], season)

  con.commit()

#inserta los datos de un jugador un partido
def insertstats(cursor, idmatch, stats, team, season):
  for x in stats:
    idplayer = getidplayer(cursor, x[0], team, season)
    if int(season[0:2]) >= 17:
      sql = "INSERT INTO stats (playerID, matchID, start, min, gls, ast, xG, npxG, xA, sh, shT, flsc, flsd, crs, tkl, int, y, r) VALUES" 
      sql = sql + "('"+str(idplayer[0][0])+"', '"+str(idmatch)+"', '"+x[1]+"', '"+str(x[5])+"', '"+str(x[6])+"', '"+str(x[7])+"', '"+str(x[8])+"', '"+str(x[9])+"',"
      sql = sql + "'"+str(x[10])+"','"+str(x[11])+"','"+str(x[12])+"','"+str(x[13])+"','"+str(x[14])+"','"+str(x[15])+"','"+str(x[16])+"','"+str(x[17])+"',"
      sql = sql + "'"+str(x[18])+"','"+str(x[19])+"')"
    else:
      sql = "INSERT INTO stats (playerID, matchID, start, min, gls, ast, sh, shT, flsc, flsd, crs, tkl, int, y, r) VALUES" 
      sql = sql + "('"+str(idplayer[0][0])+"', '"+str(idmatch)+"', '"+x[1]+"', '"+str(x[5])+"', '"+str(x[6])+"', '"+str(x[7])+"', '"+str(x[8])+"', '"+str(x[9])+"',"
      sql = sql + "'"+str(x[10])+"','"+str(x[11])+"','"+str(x[12])+"','"+str(x[13])+"','"+str(x[14])+"','"+str(x[15])+"','"+str(x[16])+"')"
    cursor.execute(sql)

#inserta los datos de un portero un partido
def insertstatsgoalkeeper(cursor, idmatch, stats, team, season):
  for x in stats:
    idplayer = getidplayer(cursor, x[0], team, season)
    sql = "UPDATE stats SET gc='"+str(x[3])+"', sav='"+str(x[4])+"', persav='"+str(x[5])+"' WHERE matchID ='"
    sql = sql + str(idmatch)+"' and playerID='"+str(idplayer[0][0])+"'"
    cursor.execute(sql)

#inserta los datos generales de un partido
def insertmatch(cursor, date, season, week, local, visit, score1, score2):
  sql = "INSERT INTO matches (date, seasonID, week, local, away, homegoals, awaygoals) VALUES ('"+date+"', '"+season+"', '"+str(week)+"', '"+local+"', '"+visit+"','"+score1+"', '"+score2+"')"
  cursor.execute(sql)
  return cursor.lastrowid

#inserta el evento de gol
def insertgoals(cursor, event, idmatch, team, season):
  for x in event:
    playerID = getidplayer(cursor, x[1], team, season)
    sql = "INSERT INTO event (matchID, playerID, type, minute) VALUES('"+str(idmatch)+"', '"+str(playerID[0][0])+"', 'goal', "+str(x[0])+")"
    cursor.execute(sql)

#inserta el evento de tarjeta roja
def insertredcard(cursor, event, idmatch, team, season):
  for x in event:
    playerID = getidplayer(cursor, x[1], team, season)
    sql = "INSERT INTO event (matchID, playerID, type, minute) VALUES('"+str(idmatch)+"', '"+str(playerID[0][0])+"', 'red', "+str(x[0])+")"
    cursor.execute(sql)

#devuelve la id de un jugador tipo 1
def getidplayer(cursor, name, team, season):
  sql = 'SELECT players.playerID FROM players JOIN playerteamseason ON players.playerID=playerteamseason.playerID and playerteamseason.seasonID="'+season+'"' 
  sql = sql+ 'and players.name="'+name+'" JOIN teams ON playerteamseason.abbrevteam=teams.abbrevteam and teams.name="'+team+'"'
  cursor.execute(sql)
  return cursor.fetchall()

#devuelve la id de un jugador tipo 2
def getidplayer2(cursor, name):
  sql = 'SELECT playerID FROM players WHERE players.name="'+name+'"'
  cursor.execute(sql)
  return cursor.fetchall()

#devuelve la abreviatura de un equipo
def getabbrevteam(cursor, name):
  sql = "SELECT abbrevteam FROM teams where name LIKE '%"+name+"%'" 
  cursor.execute(sql)
  return cursor.fetchall()

Código principal utilizado para obtener todos los jugadores e introducirlos en la bd

In [ ]:
url_season1819 = "https://fbref.com/en/comps/12/1886/schedule/2018-2019-La-Liga-Fixtures"
url_season1718 = "https://fbref.com/en/comps/12/1652/schedule/2017-2018-La-Liga-Fixtures"
url_season1617 = "https://fbref.com/en/comps/12/1547/schedule/2016-2017-La-Liga-Fixtures"
url_season1516 = "https://fbref.com/en/comps/12/1488/schedule/2015-2016-La-Liga-Fixtures"

!rm 'dbfootballdata.db'
getfiledb()
con = opendb()

teamsseason(con, url_season1819, "18-19")
teamsseason(con, url_season1718, "17-18")
teamsseason(con, url_season1617, "16-17")
teamsseason(con, url_season1516, "15-16")

closedb(con)
downloadfiledb()



Código principal utilizado para introducir datos de partidos en la bd

In [ ]:
url_season1920 = "https://fbref.com/en/comps/12/schedule/La-Liga-Fixtures"
url_season1819 = "https://fbref.com/en/comps/12/1886/schedule/2018-2019-La-Liga-Fixtures"
url_season1718 = "https://fbref.com/en/comps/12/1652/schedule/2017-2018-La-Liga-Fixtures"
url_season1617 = "https://fbref.com/en/comps/12/1547/schedule/2016-2017-La-Liga-Fixtures"
url_season1516 = "https://fbref.com/en/comps/12/1488/schedule/2015-2016-La-Liga-Fixtures"

!rm 'dbfootballdata.db'
getfiledb()
con = opendb()

seasondata(con, url_season1819, "18-19")
seasondata(con, url_season1718, "17-18")
seasondata(con, url_season1617, "16-17")
seasondata(con, url_season1516, "15-16")

closedb(con)
downloadfiledb()


Código especial para excepciones

In [ ]:
!rm 'dbfootballdata.db'
getfiledb()
con = opendb()

data = matchdata("/es/matches/887fc4e0/Getafe-Valencia-November-10-2018-La-Liga")
data[1][0]="Getafe"
data[1][1]="Valencia"
insertmatchdata(con, data, "18-19", 12)

closedb(con)
downloadfiledb()